# 🔧 环境配置和检查

## 概述
本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助您：
- 使用统一的conda环境
- 通过国内镜像源快速安装依赖
- 加速模型下载
- 检查系统配置

## 配置步骤
1. **Conda环境管理** - 激活统一的学习环境
2. **包管理器优化** - 配置pip使用清华镜像源
3. **模型下载加速** - 设置HuggingFace镜像代理
4. **系统环境诊断** - 检查硬件和软件配置


In [ ]:
# 1. 激活conda环境
%%script bash
# 初始化 conda
eval "$(conda shell.bash hook)"
conda activate flyai_agent_in_action
conda env list


In [ ]:
# 2. 设置pip 为清华源
%pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list


In [ ]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT

In [ ]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


<a href="https://colab.research.google.com/github/FlyAIBox/AIAgent101/blob/main/06-agent-evaluation/langfuse/01_02_integration_openai_structured_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 教程：使用 Langfuse 追踪 OpenAI 结构化输出

在本教程中，您将学习如何使用 Langfuse 来监控和追踪 OpenAI 的结构化输出功能。

## 什么是结构化输出？
从非结构化输入生成结构化数据是当今人工智能的核心应用场景。结构化输出使得链式大语言模型调用、UI组件生成和基于模型的评估更加可靠。[结构化输出](https://openai.com/index/introducing-structured-outputs-in-the-api/) 是 OpenAI API 的一项新功能，它基于 JSON 模式和函数调用构建，能够在模型输出中强制执行严格的数据结构模式。

**核心优势：**
- 🎯 **可靠性**：确保模型输出严格遵循预定义的数据结构
- 🔗 **链式调用**：使多个模型调用之间的数据传递更加稳定
- 🎨 **UI生成**：为前端组件提供标准化的数据格式
- 📊 **模型评估**：便于对模型输出进行自动化评估和分析

## 如何在 Langfuse 中追踪结构化输出？
如果您使用 OpenAI Python SDK，可以使用 [Langfuse 的直接替换方案](https://langfuse.com/integrations/model-providers/openai-py) 来获得完整的日志记录，只需更改导入语句即可。通过这种方式，您可以在 Langfuse 中监控 OpenAI 生成的结构化输出。

**导入方式对比：**
```diff
- import openai                    # 原始导入方式
+ from langfuse.openai import openai  # Langfuse 集成导入

其他可选导入方式：
+ from langfuse.openai import OpenAI, AsyncOpenAI, AzureOpenAI, AsyncAzureOpenAI
```

**重要说明：** 仅需修改导入语句，其余代码保持不变，即可自动获得完整的调用追踪和监控功能。



## 步骤 1：初始化 Langfuse
使用您从 Langfuse UI 项目设置中获取的 [API 密钥](https://langfuse.com/faq/all/where-are-langfuse-api-keys) 来初始化 Langfuse 客户端，并将它们添加到您的环境变量中。

**配置说明：**
- 🔑 **API 密钥**：从 Langfuse 控制台获取公钥和私钥
- 🌍 **服务器地址**：选择合适的区域服务器（欧盟或美国）
- 🤖 **OpenAI 密钥**：用于调用 OpenAI 的 API 服务

In [1]:
# 📥 安装必要的依赖包
# 这个命令会安装两个核心包：
# 1. langfuse - 可观测性平台的核心库
# 2. openai - OpenAI官方SDK

%pip install langfuse==3.3.0 openai==1.107.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.0/951.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.108.0
    Uninstalling openai-1.108.0:
      Successfully uninstalled openai-1.108.0


In [2]:
# 🔐 环境变量配置 - 安全存储敏感信息
# 环境变量是存储API密钥等敏感信息的最佳实践
# 避免在代码中硬编码密钥，防止泄露

import os, getpass

def _set_env(var: str):
    """
    安全地设置环境变量
    如果环境变量不存在，会提示用户输入
    使用getpass模块隐藏输入内容，防止密码泄露
    """
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# 🤖 OpenAI API 配置
# OpenAI API密钥：从 https://platform.openai.com/api-keys 获取
# 这是调用GPT模型必需的认证信息
_set_env("OPENAI_API_KEY")

# API代理地址：如果你使用第三方代理服务（如国内代理）
# 示例：https://api.apiyi.com/v1
# 如果直接使用OpenAI官方API，可以留空
_set_env("OPENAI_BASE_URL")

# 🌐 Langfuse 配置
# Langfuse是一个可观测性平台，需要注册账户获取密钥
# 注册地址：https://cloud.langfuse.com

# 公开密钥：用于标识你的项目
_set_env("LANGFUSE_PUBLIC_KEY")

# 秘密密钥：用于认证，请妥善保管
_set_env("LANGFUSE_SECRET_KEY")

# 服务器地址：选择离你最近的区域
# 🇪🇺 欧盟区域(推荐) https://cloud.langfuse.com
# 🇺🇸 美国区域 https://us.cloud.langfuse.com
_set_env("LANGFUSE_HOST")

# 💡 初学者提示：
# 1. 环境变量存储在操作系统中，重启后需要重新设置
# 2. 生产环境中建议使用.env文件或云服务配置
# 3. 永远不要在代码中硬编码API密钥！

OPENAI_API_KEY: ··········
OPENAI_BASE_URL: ··········
LANGFUSE_PUBLIC_KEY: ··········
LANGFUSE_SECRET_KEY: ··········
LANGFUSE_HOST: ··········


## 步骤 2：数学辅导示例

在这个示例中，我们将构建一个数学辅导工具，它将解决数学问题的步骤输出为结构化对象数组。

**应用场景：**
- 📚 **教育应用**：为学生提供逐步解题指导
- 🎯 **个性化学习**：用户可以按自己的节奏逐步学习解题过程
- 🔍 **步骤追踪**：每个解题步骤都可以单独显示和分析
- 💡 **交互式学习**：支持用户在任意步骤暂停和思考

这种设置对于需要将每个步骤单独显示的应用程序非常有用，允许用户按照自己的节奏逐步学习解题过程。

（示例改编自 [OpenAI cookbook](https://cookbook.openai.com/examples/structured_outputs_intro)）

**重要提示：** 虽然 OpenAI 也通过其测试版 API (`client.beta.chat.completions.parse`) 提供结构化输出解析功能，但这种方法目前不允许设置 Langfuse 特定的属性，如 `name`、`metadata`、`userId` 等。请使用下面描述的标准 `client.chat.completions.create` 方法配合 `response_format` 参数的方式。

**两种方法对比：**
- ✅ **推荐方式**：`client.chat.completions.create` + `response_format` - 支持完整的 Langfuse 追踪功能
- ⚠️ **限制方式**：`client.beta.chat.completions.parse` - 功能受限，无法设置追踪属性

In [3]:
# 使用 Langfuse 的直接替换方案，仅通过更改导入即可获得完整的日志记录功能
# 这样，您就可以在 Langfuse 中监控 OpenAI 生成的结构化输出
from langfuse.openai import OpenAI
import json  # 用于处理 JSON 数据的标准库

# 指定要使用的 OpenAI 模型
# gpt-4o-2024-08-06 是支持结构化输出的最新模型版本
openai_model = "gpt-4o-2024-08-06"

# 创建 OpenAI 客户端实例
# 这个客户端会自动集成 Langfuse 的追踪功能
client = OpenAI()

在 `response_format` 参数中，您现在可以通过 `json_schema` 提供 JSON Schema。当使用 `response_format` 并设置 `strict: true` 时，模型的输出将严格遵循提供的模式。

**关键概念解释：**
- 📋 **JSON Schema**：定义数据结构的标准格式，类似于数据的"蓝图"
- 🔒 **strict: true**：启用严格模式，确保输出100%符合定义的结构
- ✅ **结构保证**：模型输出将始终包含所需的字段和数据类型

函数调用保持类似的方式，但通过新的参数 `strict: true`，您现在可以确保为函数提供的模式得到严格遵循。

In [4]:
# 定义数学辅导的系统提示词
# 这个提示词告诉模型如何扮演数学老师的角色
math_tutor_prompt = '''
    你是一个有用的数学辅导老师。你将收到一个数学问题，
    你的目标是输出逐步解决方案以及最终答案。
    对于每个步骤，只需提供输出作为方程式，使用解释字段详细说明推理过程。
'''

def get_math_solution(question):
    """
    获取数学问题的结构化解决方案

    参数:
        question (str): 要解决的数学问题

    返回:
        包含逐步解决方案的结构化响应
    """
    # 调用 OpenAI API 创建聊天完成请求
    # 使用结构化输出确保返回格式符合预定义的 JSON Schema
    response = client.chat.completions.create(
    model = openai_model,  # 指定使用的模型版本
    messages=[
        {
            # 系统消息：定义AI助手的角色和行为规范
            "role": "system",
            "content": math_tutor_prompt
        },
        {
            # 用户消息：包含具体的数学问题
            "role": "user",
            "content": question
        }
    ],
    # 定义响应格式 - 这是结构化输出的核心配置
    response_format={
        # 指定使用 JSON Schema 来定义输出结构
        "type": "json_schema",
        "json_schema": {
            # Schema 的名称，用于标识这个特定的数据结构
            "name": "math_reasoning",
            # 定义具体的数据结构模式
            "schema": {
                # 根对象类型
                "type": "object",
                # 定义对象的属性
                "properties": {
                    # steps: 解题步骤数组
                    "steps": {
                        "type": "array",  # 数组类型
                        # 数组中每个元素的结构
                        "items": {
                            "type": "object",  # 每个步骤是一个对象
                            "properties": {
                                # 解释字段：说明这一步的推理过程
                                "explanation": {"type": "string"},
                                # 输出字段：这一步的数学表达式或结果
                                "output": {"type": "string"}
                            },
                            # 必需字段：每个步骤都必须包含解释和输出
                            "required": ["explanation", "output"],
                            # 不允许额外属性，确保结构严格
                            "additionalProperties": False
                        }
                    },
                    # final_answer: 最终答案
                    "final_answer": {"type": "string"}
                },
                # 根对象的必需字段
                "required": ["steps", "final_answer"],
                # 不允许额外属性
                "additionalProperties": False
            },
            # 启用严格模式：确保输出100%符合定义的结构
            "strict": True
        }
    }
    )

    # 返回模型生成的消息内容
    # choices[0] 表示选择第一个（通常也是唯一的）响应
    return response.choices[0].message

In [5]:
# 使用示例问题进行测试
# 这是一个一元一次方程，适合演示逐步解题过程
question = "如何解这个方程：8x + 7 = -23"

# 调用我们定义的函数获取结构化解决方案
result = get_math_solution(question)

# 打印原始的 JSON 格式响应内容
# 这展示了模型返回的结构化数据
print("原始 JSON 响应:")
print(result.content)

原始 JSON 响应:
{"final_answer":"x = -3.75","steps":[{"explanation":"首先，我们从方程两边减去7，以便将8x独立出来。","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"简化方程，我们得到：8x = -30。","output":"8x = -30"},{"explanation":"接下来，我们将方程的两边都除以8，以便求出x的值。","output":"x = -30 / 8"},{"explanation":"通过计算-30 ÷ 8，我们得到x的值是-3.75。","output":"x = -3.75"}]}


In [6]:
# 逐步打印解题结果 - 展示如何处理结构化输出

# 将 JSON 字符串解析为 Python 字典
# 这展示了结构化输出的一个重要优势：数据格式可预测且易于处理
result = json.loads(result.content)

# 提取解题步骤数组和最终答案
steps = result['steps']
final_answer = result['final_answer']

# 遍历每个解题步骤并格式化输出
for i in range(len(steps)):
    # 打印步骤编号和解释
    print(f"步骤 {i+1}: {steps[i]['explanation']}\n")
    # 打印该步骤的数学表达式
    print(steps[i]['output'])
    print("\n")

# 打印最终答案
print("最终答案:\n\n")
print(final_answer)

步骤 1: 首先，我们从方程两边减去7，以便将8x独立出来。

8x + 7 - 7 = -23 - 7


步骤 2: 简化方程，我们得到：8x = -30。

8x = -30


步骤 3: 接下来，我们将方程的两边都除以8，以便求出x的值。

x = -30 / 8


步骤 4: 通过计算-30 ÷ 8，我们得到x的值是-3.75。

x = -3.75


最终答案:


x = -3.75


## 步骤 3：在 Langfuse 中查看您的追踪记录

现在您可以在 Langfuse 中查看追踪记录和 JSON Schema。

**Langfuse 追踪功能的价值：**
- 📊 **完整记录**：记录每次 API 调用的详细信息
- 🔍 **结构分析**：可视化展示 JSON Schema 和实际输出
- ⏱️ **性能监控**：追踪响应时间和 token 使用情况
- 🐛 **调试支持**：帮助识别和解决问题
- 📈 **使用统计**：分析模型使用模式和成本

[Langfuse 中的示例追踪记录](https://cloud.langfuse.com/project/cmequpe0j00euad07w6wrvkzg/traces?peek=058c85fac3a31c8fcad5291467b92633&timestamp=2025-09-23T02%3A49%3A38.363Z)

![在 Langfuse UI 中查看示例追踪记录](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509231059860.png)
**图片说明：** 上图展示了 Langfuse 界面中的追踪记录，您可以看到：
- 🔄 **调用流程**：完整的 API 调用链路
- 📝 **输入输出**：系统提示、用户问题和模型响应
- 📊 **结构化数据**：JSON Schema 和实际输出的对比
- ⏱️ **性能指标**：响应时间、token 消耗等关键指标

## 替代方案：使用 SDK 的 `parse` 辅助方法

新版本的 SDK 添加了 `parse` 辅助方法，允许您使用自己的 Pydantic 模型而无需定义 JSON Schema。

**Pydantic 方法的优势：**
- 🐍 **Python 原生**：使用 Python 类定义数据结构，更符合 Python 开发习惯
- 🔧 **类型检查**：提供更好的 IDE 支持和类型提示
- 📝 **简洁语法**：相比 JSON Schema，代码更简洁易读
- ✅ **自动验证**：Pydantic 提供强大的数据验证功能

**注意：** 这种方法目前在 Langfuse 追踪中的功能可能有限。

In [7]:
# 导入 Pydantic 用于定义数据模型
# Pydantic 是 Python 中最流行的数据验证库
from pydantic import BaseModel

# 定义数学推理的数据模型
class MathReasoning(BaseModel):
    """数学推理结果的数据模型"""

    # 内嵌类定义单个解题步骤的结构
    class Step(BaseModel):
        """单个解题步骤的数据模型"""
        explanation: str  # 解释说明：描述这一步的推理过程
        output: str       # 输出结果：这一步的数学表达式或计算结果

    # 主模型的字段定义
    steps: list[Step]    # 解题步骤列表：包含所有解题步骤
    final_answer: str    # 最终答案：问题的最终解答

def get_math_solution(question: str):
    """
    使用 Pydantic 模型获取数学问题的结构化解决方案

    参数:
        question (str): 要解决的数学问题

    返回:
        包含解析后的结构化数据的响应
    """
    # 使用 beta API 的 parse 方法
    # 这个方法会自动将 Pydantic 模型转换为 JSON Schema
    response = client.beta.chat.completions.parse(
        model=openai_model,  # 指定模型
        messages=[
            # 系统消息：定义AI的角色
            {"role": "system", "content": math_tutor_prompt},
            # 用户消息：具体的数学问题
            {"role": "user", "content": question},
        ],
        # 直接传入 Pydantic 模型类，SDK 会自动处理转换
        response_format=MathReasoning,
    )

    # 返回响应消息
    # 注意：使用 .parsed 属性可以直接获取解析后的 Pydantic 对象
    return response.choices[0].message

In [8]:
# 调用函数并获取解析后的 Pydantic 对象
# .parsed 属性直接返回类型化的 Python 对象，而不是 JSON 字符串
result = get_math_solution(question).parsed

# 打印解题步骤 - 这里 result.steps 是 Step 对象的列表
print("解题步骤:")
print(result.steps)

# 打印最终答案 - 直接访问对象属性
print("\n最终答案:")
print(result.final_answer)

解题步骤:
[Step(explanation='首先，我们要将等式中的常数项移到方程的另一边，以便能够隔离变量x。我们通过从等式两边减去7来做到这一点。', output='8x + 7 - 7 = -23 - 7'), Step(explanation='计算等式两边的简化结果。左边的7减去7得到0，而右边的-23减去7得到-30。', output='8x = -30'), Step(explanation='现在，我们要通过除以系数8来使x单独成为方程中的一项。我们通过在两边除以8来做到这一点。', output='8x/8 = -30/8'), Step(explanation='计算等式的结果，x等于-30除以8。', output='x = -3.75')]

最终答案:
x = -3.75


## 在 Langfuse 中查看您的追踪记录

现在您可以在 Langfuse 中查看追踪记录和您提供的 Pydantic 模型。

**Pydantic 方法的追踪特点：**
- 🏗️ **模型结构**：显示 Pydantic 模型的类定义和字段类型
- 🔄 **自动转换**：展示从 Pydantic 到 JSON Schema 的自动转换过程
- 📊 **类型信息**：保留完整的 Python 类型注解信息

[Langfuse 中的 Pydantic 示例追踪记录](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/59c4376a-c8eb-4ecb-8780-2f028b87e7eb)

![在 Langfuse UI 中查看 Pydantic 方法的追踪记录](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202509231103540.png)

**图片说明：** 上图展示了使用 Pydantic 方法时的 Langfuse 追踪界面，您可以看到：
- 🐍 **Pydantic 模型**：完整的类定义和字段类型信息
- 🔄 **Schema 转换**：自动生成的 JSON Schema
- 📝 **类型安全**：强类型的数据结构和验证
- ⚡ **开发效率**：更简洁的代码和更好的 IDE 支持